In [1]:
inputfile = 'AirQualityData.csv'

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(inputfile, encoding = "ISO-8859-1")

In [4]:
df = df.drop(['stn_code', 'sampling_date', 'agency', 'type', 'location_monitoring_station','pm2_5', 'spm'], axis = 1)

In [5]:
df.columns.values

array(['state', 'location', 'so2', 'no2', 'rspm', 'date'], dtype=object)

In [6]:
years = ['2013','2014']
df = df[df["date"].str.contains('|'.join(years), na=False)]

In [7]:
# df2 = pd.read_csv('Air_Quality_MapFile.csv')

# df2.drop(df2[df2['Year'] < 2014 ].index , inplace=True)
# df2 = df2.drop(['Year', 'zscore', 'spot'], axis = 1)
# df2

# state = set(df['state'])
# state2 = set(df2['State'])
# state - state2

In [8]:
df.loc[:, 'date'] = df['date'].str.split('-', expand=True)[0]

In [9]:
df['state'] = df['state'].replace(['Uttaranchal'],'Uttarakhand')
df['state'] = df['state'].replace(['Telangana'],'Andhra Pradesh')

In [10]:
df = df.groupby(['state','location', 'date']).mean()

In [11]:
# Snippets for checking null values (set year as 2014)
# df.sort_values(by = ['state','location','date'], ascending=[True,True,False])
# df.isnull().sum()
# df[df['so2'].isnull()]
# df[df['no2'].isnull()]

In [12]:
# Only this row has NA in no2 and so2 in 2014, rspm is present for all cities in 2014
df.loc['Andhra Pradesh','Eluru','2014']['so2'] = df.loc['Andhra Pradesh','Eluru','2013']['so2']
df.loc['Andhra Pradesh','Eluru','2014']['no2'] = df.loc['Andhra Pradesh','Eluru','2013']['no2']
df.loc['Andhra Pradesh','Eluru','2014']

so2       5.459459
no2      15.189189
rspm    121.482143
Name: (Andhra Pradesh, Eluru, 2014), dtype: float64

In [13]:
df = df.reset_index(level=['state','location','date'])

In [14]:
df = df[df['date'].str.contains('2014', na=False)]
df.columns

Index(['state', 'location', 'date', 'so2', 'no2', 'rspm'], dtype='object')

In [15]:
df = df.drop(['date'], axis = 1)

In [16]:
# For normalisation if required in future
# df = (df-df.min())/(df.max()-df.min())

In [17]:
df['Mean-Pollutant-Concentration'] = df.mean(axis=1)

In [18]:
# Mean-Pollutant-Concentration
df = df.drop(['so2', 'no2', 'rspm'], axis = 1)

In [19]:
df = df.sort_values(by = ['state','Mean-Pollutant-Concentration'], ascending=[True,False])

In [20]:
# ADDED MPC AND XCO YCO FROM Air_Quality_MapFile.csv

df2 = pd.read_csv('Zscore_MapFile.csv')
df2 = df2[df2['Year'] == 2014 ]
# df2.drop(df2[df2['Year'] < 2014 ].index , inplace=True)
df2 = df2.drop(['Year', 'zscore', 'spot'], axis = 1)

# Check if inner join is safe (must give empty set)
state = set(df['state'])
state2 = set(df2['State'])
state - state2

df2.rename(columns = {'State':'state', 'Mpc':'state-Mpc'}, inplace = True)

df = pd.merge(df, df2)

In [21]:
df = df.groupby('state').head(5)

In [22]:
df.to_csv("PollutedCities.csv", index=False)

In [23]:
# set(df['state'])